In [35]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import neurokit2 as nk
from collections import Counter
from micromlgen import port
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, cross_validate
from mlxtend.feature_selection import ExhaustiveFeatureSelector

In [18]:
folder_path = "Labeled"

# Cari file dengan pola nama tertentu
csv_files = [f for f in os.listdir(folder_path) if f.startswith("ecg_") and f.endswith("_labeled.csv")]

print("File yang ditemukan:", csv_files)

File yang ditemukan: ['ecg_Adimas_labeled.csv', 'ecg_Aldo_labeled.csv', 'ecg_Anugrah_labeled.csv', 'ecg_Aufa_labeled.csv', 'ecg_Bagja_labeled.csv', 'ecg_Danzel_labeled.csv', 'ecg_Edo_labeled.csv', 'ecg_Eldon_labeled.csv', 'ecg_Faiz_labeled.csv', 'ecg_Fathan_labeled.csv', 'ecg_Fitra_labeled.csv', 'ecg_Ghifari_labeled.csv', 'ecg_Gilang_labeled.csv', 'ecg_Haikal_labeled.csv', 'ecg_Jepi_labeled.csv', 'ecg_Khairian_labeled.csv', 'ecg_Panji_labeled.csv', 'ecg_Rafi_labeled.csv', 'ecg_Rama_labeled.csv', 'ecg_Randy_labeled.csv', 'ecg_Reza_labeled.csv']


In [ ]:
def majority_voting(label_window):
    label_count = Counter(label_window)
    if len(label_count) == 0:
        return None  
    
    majority_label = label_count.most_common(1)[0][0]
    return majority_label

sampling_rate = 100  
window_duration = 180  
window_size = window_duration * sampling_rate  
overlap_percentage = 0.95  
overlap_size = int(window_size * overlap_percentage)  
all_hrv_features = []

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    data = pd.read_csv(file_path)
    ecg_signal = data['ECG Value']
    labels = data['Label']  
    hrv_features_subj = []  
    
    for start in range(0, len(ecg_signal) - window_size, window_size - overlap_size):
        end = start + window_size
        ecg_window = ecg_signal[start:end]
        label_window = labels[start:end]
        majority_label = majority_voting(label_window)
        if majority_label is None:
            continue  
        
        _, rpeaks = nk.ecg_peaks(ecg_window, sampling_rate=sampling_rate)
        
        if len(rpeaks['ECG_R_Peaks']) > 0:
            hrv_features = nk.hrv_time(rpeaks, sampling_rate=sampling_rate)
            hrv_features['Label'] = majority_label  
            hrv_features['Subject'] = file  
            hrv_features_subj.append(hrv_features)
    
    if hrv_features_subj:
        hrv_features_subj_df = pd.concat(hrv_features_subj, ignore_index=True)
        all_hrv_features.append(hrv_features_subj_df)

if all_hrv_features:
    combined_hrv_features = pd.concat(all_hrv_features, ignore_index=True)
    print(combined_hrv_features.head())
    combined_hrv_features.to_csv("hrv_time_domain.csv", index=False)
else:
    print("Tidak ada fitur HRV yang diekstraksi.")

   HRV_MeanNN   HRV_SDNN  HRV_SDANN1  HRV_SDNNI1  HRV_SDANN2  HRV_SDNNI2  \
0  815.662100  52.670823   16.042489   49.402948         NaN         NaN   
1  820.504587  49.599525    4.436394   48.892156         NaN         NaN   
2  822.798165  47.664008   12.442260   46.927787         NaN         NaN   
3  822.165899  47.971926   19.029719   45.643701         NaN         NaN   
4  822.995392  48.665192   14.454848   46.794551         NaN         NaN   

   HRV_SDANN5  HRV_SDNNI5  HRV_RMSSD   HRV_SDSD  ...  HRV_Prc20NN  \
0         NaN         NaN  36.935351  37.018531  ...        770.0   
1         NaN         NaN  36.851927  36.937105  ...        784.0   
2         NaN         NaN  38.238880  38.327266  ...        790.0   
3         NaN         NaN  37.749172  37.836745  ...        790.0   
4         NaN         NaN  37.087235  37.173123  ...        790.0   

   HRV_Prc80NN  HRV_pNN50  HRV_pNN20  HRV_MinNN  HRV_MaxNN    HRV_HTI  \
0        860.0  10.958904  54.337900      690.0      93

In [37]:
# Memuat dataset
combined_hrv_features = pd.read_csv('hrv_features_all_subjects.csv')

In [49]:
columns_to_drop = ['HRV_SDANN1','HRV_SDNNI1','HRV_SDANN2','HRV_SDNNI2','HRV_SDANN5','HRV_SDNNI5','HRV_pNN50','HRV_pNN20', 'HRV_TINN', 'Subject', 'HRV_IQRNN', 'HRV_HTI', 'HRV_MadNN', 'HRV_MCVNN']  # Sesuaikan dengan kolom yang ingin dihapus
df_hrv_final = combined_hrv_features.drop(columns=columns_to_drop)
print(df_hrv_final.head())

   HRV_MeanNN   HRV_SDNN  HRV_RMSSD   HRV_SDSD  HRV_CVNN  HRV_CVSD  \
0  815.662100  52.670823  36.935351  37.018531  0.064574  0.045283   
1  820.504587  49.599525  36.851927  36.937105  0.060450  0.044914   
2  822.798165  47.664008  38.238880  38.327266  0.057929  0.046474   
3  822.165899  47.971926  37.749172  37.836745  0.058348  0.045914   
4  822.995392  48.665192  37.087235  37.173123  0.059132  0.045064   

   HRV_MedianNN  HRV_SDRMSSD  HRV_Prc20NN  HRV_Prc80NN  HRV_MinNN  HRV_MaxNN  \
0         820.0     1.426027        770.0        860.0      690.0      930.0   
1         820.0     1.345914        784.0        860.0      690.0      930.0   
2         830.0     1.246480        790.0        860.0      690.0      930.0   
3         820.0     1.270807        790.0        860.0      690.0      930.0   
4         830.0     1.312182        790.0        860.0      690.0      930.0   

    Label  
0  Normal  
1  Normal  
2  Normal  
3  Normal  
4  Normal  


In [50]:
X = df_hrv_final.drop(columns=['Label'])
y = df_hrv_final['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

In [51]:
rf_model = RandomForestClassifier(n_estimators=5, max_depth=15, random_state=33)

rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

In [52]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8693994280266921
Classification Report:
               precision    recall  f1-score   support

      Normal       0.85      0.77      0.81       231
 Stres Berat       0.88      0.90      0.89       396
Stres Ringan       0.87      0.89      0.88       422

    accuracy                           0.87      1049
   macro avg       0.87      0.86      0.86      1049
weighted avg       0.87      0.87      0.87      1049



In [169]:
# Evaluasi menggunakan cross-validation
cv_scores = cross_val_score(rf_model, X_train, y_train, cv=10, scoring='accuracy')  # 5-fold CV

# Cetak hasil
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", cv_scores.mean())
print("Standard Deviation:", cv_scores.std())

Cross-Validation Scores: [0.85747847 0.86329808 0.8515472  0.87230709 0.85624755 0.86564826
 0.87700744 0.8715237  0.86447317 0.87269879]
Mean Accuracy: 0.8652229738164845
Standard Deviation: 0.007852744642811825


In [43]:
print(port(rf_model))

#pragma once
#include <cstdarg>
namespace Eloquent {
    namespace ML {
        namespace Port {
            class RandomForest {
                public:
                    /**
                    * Predict class for features vector
                    */
                    int predict(float *x) {
                        uint8_t votes[3] = { 0 };
                        // tree #1
                        if (x[0] <= 484.51220703125) {
                            if (x[1] <= 18.20659065246582) {
                                votes[0] += 1;
                            }

                            else {
                                votes[2] += 1;
                            }
                        }

                        else {
                            if (x[3] <= 495.0) {
                                if (x[2] <= 9.84710693359375) {
                                    if (x[5] <= 530.0) {
                                        votes[2] += 1;
                         